# Generate-Label-Masks-PNG
This code reads in the labels provided by AlphaPilot organizers AND those we manually annotated to get a mask that can be used to train segmentation networks.

In [1]:
%matplotlib inline
import json
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
#from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import glob
import time

from scipy.ndimage.measurements import label
import pandas as pd
#import keras.backend as K
from sklearn.metrics import jaccard_similarity_score
 
import time
import numpy
from PIL import Image, ImageDraw
import re
from shapely.geometry import Polygon
from pprint import pprint
import PIL.ImageDraw
from PIL import Image
import argparse
import os
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

#add a note for the class

In [2]:
#df is the dataset that we are annotating
#raw_df is the dataset that the organizers provided
#adding path for json anf image folders respectively
provided_data_file_dir='training_GT_labels_v2.json'#json provided by the organizers of the challange
our_data_file_dir='testing_leaderboard_GT_labels_v3_XY_format.json'#our annotated dataset
img_file_dir='Data_Test_Leaderboard/'#folder where images are stored

provided_df = pd.read_json(provided_data_file_dir)
df = pd.read_json(our_data_file_dir)

 
#adding a complete path for the image 
df['images']= df['External ID']
df['External ID']= [img_file_dir + u for u in df['External ID']]#iris_data_dir + new_df['parcel_id'].astype(str) + '.jpg'

# df['images']=[u.split('/',8)[8] for u in df['External ID']]
raw_df=pd.DataFrame()
raw_df['images']=list(provided_df.columns.values)
raw_df['img_path']=[img_file_dir +  u for u in raw_df['images']]

raw_df['raw_inner_poly']=[ provided_df[u][0] for u in raw_df['images']]
# raw_df.head()




#remove white spaces from image path
s=raw_df['images'][3]
print(re.sub(r"\s+", "", s))  # \s matches all white spaces


#merge the our annotations and provided annotations
df_all=pd.merge(df, raw_df, on='images')
# df_all.head(10)



keep_cols=['images', 'img_path','Label','raw_inner_poly']
df_all=df_all[keep_cols]
df_all['outer_poly']=''
df_all['inner_poly']=''
df_all['outer_x_min']=''
df_all['outer_y_min']=''
df_all['outer_x_max']=''
df_all['outer_y_max']=''
df_all['inner_x_min']=''
df_all['inner_y_min']=''
df_all['inner_x_max']=''
df_all['inner_y_max']=''
df_all['class_id']= ''

# checking the mask from the provided GT by the organizers
#processing the polygone and creating a mask
def get_mask_raw_data(img_shape, poly,display=False):
    output_mask = np.zeros(img_shape[:2], dtype=np.uint8)
    print('poly',poly)

    coords =  zip(*[iter(poly)] * 2) 
     
    mask = np.zeros(img_shape[:2], dtype=np.uint8)
    mask = PIL.Image.fromarray(mask)
    draw = PIL.ImageDraw.Draw(mask)
    x = []
    y = []
    polygons = []

    for pt in coords:
        a = (int(pt[0]), int(pt[1]))
        polygons.append(a)
        init_x = (int(pt[0]))
        init_y = (int(pt[1]))

        x.append(init_x)
        y.append(init_y)
    proc_polygons=np.vstack((x,y)).T
    x,y,w,h = cv2.boundingRect(proc_polygons) 
    draw.polygon(xy=polygons, outline=1, fill=1)
    mask = np.array(mask, dtype=bool)
    if display:
        plt.imshow(mask)
        plt.show
    return mask

for i in range(len(df)):#len(df)):

    img=cv2.imread(df_all['img_path'][i])
    row=img.shape[0]
    col=img.shape[1]
    img_shape = (row,col)
    #outer_poly=df['Label'][i] 
    inner_poly=df_all['raw_inner_poly'][i]
    if inner_poly:
        print('raw_inner_poly',inner_poly)
        inner_mask=get_mask_raw_data(img_shape,inner_poly,display=False)
        inner_mask.dtype='uint8'
#         plt.imshow(cv2.bitwise_and(img,img,mask=inner_mask))
#         plt.show
    else:
        print('polygon data corruption detected for index:', i)

        
        
#convert the polygons into a format that can be convereted to bounding boxes 
from tqdm import tqdm
#preprocessing our_dataset

def convert_coordinates(poly):
    proc_poly=poly[0]['geometry']
    nums =  re.findall(r'\d+(?:\.\d*)?', proc_poly.rpartition(',')[0])
    coords =  zip(*[iter(nums)] * 2)
    polygons = []
    for pt in coords:
        a = (int(pt[0]), int(pt[1]))
        polygons.append(a)
    
    return polygons 
def convert_coordinates_raw(polygon):
    poly=[]
    polygon =  zip(*[iter(polygon)] * 2)
    for p in polygon:
        poly.append(p)
    return poly
def get_bbox(polygon):
    polygon=polygon[0]['geometry']
    polygon=(polygon)

    polygon =  re.findall(r'\d+(?:\.\d*)?', polygon)
    polygon =  zip(*[iter(polygon)] * 2)
    x = []
    y = []
    for pt in polygon:
        init_x = (int(pt[0]))
        init_y = (int(pt[1]))

        x.append(init_x)
        y.append(init_y)
    polygons=np.vstack((x,y)).T
   
    x,y,w,h = cv2.boundingRect(polygons)
    x_min=x
    y_min=y
    x_max=x+w
    y_max=y+h
    return x_min,y_min,x_max,y_max
    #return x_min,x_max,y_min,y_max
def get_bbox_raw_data(polygon):
    polygon =  zip(*[iter(polygon)] * 2)
    x = []
    y = []
    for pt in polygon:
 
        init_x = (int(pt[0]))
        init_y = (int(pt[1]))

        x.append(init_x)
        y.append(init_y)
    polygons=np.vstack(((x,y))).T
    
    x,y,w,h = cv2.boundingRect(polygons)
    x_min=x
    y_min=y
    x_max=x+w
    y_max=y+h
    return x_min,y_min,x_max,y_max
    #return x_min,x_max,y_min,y_max
for i in tqdm(range(0,len(df_all))):
    #remove white spaces from image path
   
    df_all['img_path'][i]=re.sub(r"\s+", "", df_all['img_path'][i]) # \s matches all white spaces
    if df_all['raw_inner_poly'][i]:
        if df_all['Label'][i] != 'Skip':
            outer_poly=df_all['Label'][i]['Outer Border']
            #if len(outer_poly) !=1:
 
            inner_poly=df_all['raw_inner_poly'][i]#df['Label'][i]['inner flyable area']
            if inner_poly and outer_poly:
                df_all['outer_poly'][i] = convert_coordinates(outer_poly)
                df_all['inner_poly'][i] = convert_coordinates_raw(inner_poly)
                    #df['inner_poly'][i] = convert_coordinates(inner_poly)

                df_all['outer_x_min'][i],df_all['outer_y_min'][i], df_all['outer_x_max'][i],df_all['outer_y_max'][i]=get_bbox(outer_poly)
                df_all['inner_x_min'][i],df_all['inner_y_min'][i], df_all['inner_x_max'][i],df_all['inner_y_max'][i]=get_bbox_raw_data(inner_poly)
                    #df_all['outer_x_min'][i],df_all['outer_x_max'][i], df_all['outer_y_min'][i],df_all['outer_y_max'][i]=get_bbox(outer_poly)
                    #df_all['inner_x_min'][i],df_all['inner_x_max'][i], df_all['inner_y_min'][i],df_all['inner_y_max'][i]=get_bbox_raw_data(inner_poly) 
        else:
            print('corrupt data detected for index:', i )
            continue
    #df['outer_poly']= h
   #df['inner_poly']= convert_coordinates(inner_poly)


df_all.reset_index(drop=True, inplace=True)
# df_all.head()



#keep only filed required by yolov3. Divide the dataframe into outer and inner bounding boxes and then assign a class to each one of them 
keep_cols_outer=['img_path', 'outer_x_min', 'outer_y_min', 'outer_x_max', 'outer_y_max','outer_poly','class_id']
keep_cols_inner=['img_path', 'inner_x_min', 'inner_y_min', 'inner_x_max', 'inner_y_max', 'inner_poly','class_id']
df_outer=df_all[keep_cols_outer]
df_inner=df_all[keep_cols_inner]
df_outer['class_id']=0
df_inner['class_id']=1
df_enet=pd.merge(df_outer, df_inner, on='img_path')
df_enet.head()


IMG_4609.JPG


IndexError: index out of bounds

In [3]:
import imageio
import shutil

# testing annotation all together 
def get_mask(img_shape, poly,display=False):
     

    mask = np.zeros(img_shape[:2], dtype=np.uint8)
    mask = PIL.Image.fromarray(mask)
    draw = PIL.ImageDraw.Draw(mask)
    polygons = []
    for pt in poly:
        a = (int(pt[0]), int(pt[1]))
        polygons.append(a)
    
    draw.polygon(xy=polygons, outline=1, fill=1)
    mask = np.array(mask, dtype=bool)
    if display:
        plt.imshow(mask)
        plt.show
    return mask

# #getting the final mask
for i in range(0,len(df_enet)): # use one image only for testing
    print(i)
    img=cv2.imread(df_enet['img_path'][i])
    row=img.shape[0]
    col=img.shape[1]
    img_shape = (row,col)
    
    outer_poly=df_enet['outer_poly'][i] 
    inner_poly=df_enet['inner_poly'][i]
    
    if len(outer_poly) < 2:
        print('gt annotation error has been detected in img ID:', i)
        print('skipping img')
        
    else: 
        outer_mask=get_mask(img_shape,outer_poly,display=False)
        inner_mask=get_mask(img_shape,inner_poly,display=False)
        inner_mask.dtype='uint8'
        outer_mask.dtype='uint8'
        
        final_mask=cv2.subtract(outer_mask,inner_mask)
        
        # Get image name
        img_path = os.path.basename(df_enet['img_path'][i])
        img_name = os.path.splitext(img_path)[0]
        imageio.imsave(os.path.join('datasets/train/labels', img_name + '-label.png'), final_mask)
        
#         img = imageio.imread(df_enet['img_path'][i])
#         imageio.imsave(os.path.join('datasets/train/images', img_name + '-rgb.png'), img)
        
        shutil.copy(df_enet['img_path'][i], os.path.join('datasets/train/images', img_name + '-rgb.JPG'))
        
        

NameError: name 'df_enet' is not defined